In [2]:
# Import libraries
import MySQLdb
import pandas as pd
import datetime
import os

In [3]:
# read the password
p = pd.read_csv('/home/ubuntu/Spitishop/Spitishop_RE/p.txt', sep=" ", header=None).iloc[0,0]

print("Fetching data")
# connect to database
conn = MySQLdb.connect("195.201.212.160","infilab",p,"spitishop_redesign_2019", charset='utf8')


#df = pd.read_sql(q, conn)

# read the products that are active
df_ps_product_shop = pd.read_sql("select id_product, active from ps_product_shop where active=1", conn)

# read the availability table - ps_stock_available
#  and (quantity>0 or out_of_stock=1)
#df_ps_stock_available = pd.read_sql("select id_product, quantity, out_of_stock from ps_stock_available where id_shop=1", conn)

# Read the tables about categories
df_ps_category = pd.read_sql("select id_category, level_depth from ps_category", conn)
df_ps_category_product = pd.read_sql("select id_product, id_category from ps_category_product", conn)
# Read the table with the category names
df_ps_category_lang = pd.read_sql("select id_category, name from ps_category_lang where id_lang=2 and id_shop=1", conn)

# Close connection
conn.close()

print("Data fetched")

Fetching data
Data fetched


In [4]:
# merge active products with categories
df = df_ps_product_shop.merge(df_ps_category_product, left_on='id_product', right_on='id_product', how='left')
# add the level depth of the category
df2 = df.merge(df_ps_category, left_on='id_category', right_on='id_category', how='left')
# add the category name
df3 = df2.merge(df_ps_category_lang, left_on='id_category', right_on='id_category', how='left')
# keep only categories with level depth=2
df4 = df3[df3.level_depth==2]
# keep only uniques
df4 = df4.drop_duplicates(subset=['id_product'], keep='first')
# print the unique categories
df4.name.unique()

array(['Βρεφικά', 'Κρεβατοκάμαρα', 'Μπάνιο', 'Παιδικά', 'Σαλόνι',
       'Κουρτίνες', 'Κουζίνα - Τραπεζαρία', 'Χαλιά', 'Μόδα Γάμου',
       'Προώθηση', 'Θαλάσσης', 'Διακόσμηση', 'Χριστουγεννιάτικα',
       'Παιχνίδια', 'Επαγγελματικός Εξοπλισμός', 'Φωτιστικά',
       'Δώρα για νεογέννητο', 'Κήπος', 'Πασχαλινά',
       'Υγιεινή - Μάσκες Προστασίας', 'Clearance'], dtype=object)

In [5]:
df4.name.nunique()

21